# 07_d. Limpieza y Estandarización de Datos.

En esta sesión nos adentramos en el corazón del análisis de datos: la **limpieza y preparación**.

Se dice que un analista de datos pasa hasta el 80% de su tiempo limpiando y preparando los datos. ¿Por qué? Porque los datos del mundo real son "sucios": están incompletos, son inconsistentes o contienen errores. Un modelo de análisis o aprendizaje automático es como un motor de alta precisión: si le echas combustible sucio (datos sucios), funcionará mal, se atascará o dará resultados incorrectos. 

En este notebook, cubriremos íntegramente el **Resultado de Aprendizaje 2 (RA 2)**: *Limpia y estandariza lotes de datos de forma lógica y eficiente para su tratamiento posterior de acuerdo al problema a resolver*.

---

## Bloque 1: Análisis y Limpieza de Datos

**Objetivos del Bloque:**
1.  **Analizar** un conjunto de datos recién cargado para identificar problemas (`RA 2.a`).
2.  **Escribir código** para gestionar los problemas más comunes: datos faltantes y duplicados (`RA 2.c`).

### Análisis de los datos leídos.

El primer paso después de cargar datos (`pd.read_csv`, `pd.read_excel`, etc.) no es analizarlos, sino **analizar su estado**. Debemos responder a preguntas como:

* ¿Están todos los datos presentes?
* ¿Tienen los datos el formato (tipo) que espero?
* ¿Hay valores que no tienen sentido?
* ¿Hay información duplicada?

Para este bloque, usaremos un conjunto de datos clásico: el del **Titanic**. Es un set de datos fantástico para aprender a limpiar. Primero, lo cargaremos usando la librería `seaborn`, que incluye algunos datasets de ejemplo.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns # Seaborn es una librería de visualización que trae datasets de ejemplo

# Cargamos el dataset 'titanic'
df_titanic = sns.load_dataset('titanic')

# Establecemos 'pandas' para que muestre más filas y columnas (opcional, pero útil)
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

#### 1. Vistazo Inicial: `.head()`, `.tail()`, `.shape`
Estos métodos nos dan una idea de la estructura y el contenido.

In [2]:
print("Forma del DataFrame (filas, columnas):", df_titanic.shape)
print("\n--- Primeras 5 filas ---")
df_titanic.head()

Forma del DataFrame (filas, columnas): (891, 15)

--- Primeras 5 filas ---


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


#### 2. Resumen Técnico: `.info()`
Este es el comando **más importante** para un primer análisis. Nos dice:
* El número total de filas (entradas).
* El número de columnas.
* El nombre de cada columna.
* El número de valores **no nulos** en cada columna (¡clave para detectar datos faltantes!).
* El tipo de dato (`Dtype`) de cada columna (`object` suele ser texto).

In [3]:
df_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


**Análisis de `.info()`:**
* Tenemos 891 filas (entradas).
* **¡Problema!** La columna `age` (edad) solo tiene 714 valores no nulos (891 - 714 = 177 valores faltantes).
* **¡Gran Problema!** La columna `deck` (cubierta) solo tiene 203 valores no nulos. La mayoría están vacíos.
* `embarked` (puerto de embarque) también tiene 2 valores faltantes.

#### 3. Resumen Estadístico: `.describe()`
Nos da estadísticas descriptivas (media, mediana, min, max...) para las **columnas numéricas**[cite: 1943, 2341].

In [4]:
df_titanic.describe()

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


**Análisis de `.describe()`:**
* Nos confirma que `age` tiene un `count` de 714.
* Vemos que la media de edad es de 29.7 años.
* `survived` (superviviente) es 0 o 1, pero la media es 0.38, lo que nos dice que el 38% de las personas en este dataset sobrevivió.
* ¿Tiene sentido que la `pclass` (clase) mínima sea 1 y la máxima 3? Sí.
* ¿Tiene sentido que la `fare` (tarifa) mínima sea 0? Quizás era tripulación o un invitado. Es algo a investigar.

Para ver un resumen de las **columnas no numéricas** (tipo `object`), podemos usar `include=['object']`:

In [5]:
df_titanic.describe(include=['object'])

,sex,embarked,who,embark_town,alive
count,891,889,891,889,891
unique,2,3,3,3,2
top,male,S,man,Southampton,no
freq,577,644,537,644,549


**Análisis:**
* `sex`: Tiene 2 valores únicos ('male', 'female'). El más frecuente ('top') es 'male' (577 veces).
* `embarked`: Tiene 3 valores únicos. El más frecuente es 'S' (Southampton).

### Escribiendo código que permite limpiar... datos.

Una vez analizados los problemas, ¡vamos a solucionarlos! Empezaremos por los datos faltantes y los duplicados.

### Aclaración sobre datos faltantes/ausentes (NA: Not Available)   

En aplicaciones de estadística, los datos NA pueden ser o bien datos que no existen o que existen pero no se han observado (debido a problemas con la recogida de datos, por ejemplo). Al limpiar datos para su análisis, suele ser importante analizar también los datos que no están, para identificar problemas en la recogida de datos o posibles desviaciones en los datos producidas por datos faltantes.

In [6]:
import pandas as pd
import numpy as np

string_data = pd.Series(["aardvark", np.nan, None,"avocado"]) # Ejemplo de datos de tipo string con valores NA
print(string_data)
string_data.isna()

0    aardvark
1         NaN
2        None
3     avocado
dtype: object


0    False
1     True
2     True
3    False
dtype: bool

In [7]:
float_data = pd.Series([1, 2, np.nan, 4, None]) # Ejemplo de datos de tipo float con valores NA
print(float_data)
float_data.isna()

0    1.0
1    2.0
2    NaN
3    4.0
4    NaN
dtype: float64


0    False
1    False
2     True
3    False
4     True
dtype: bool

#### 1. Limpieza de Datos Faltantes (NaN)

En Pandas, los datos faltantes se representan con `NaN` (Not a Number). Como vimos con `.info()`, tenemos valores faltantes en `age`, `deck` y `embarked`.

Tenemos dos estrategias principales: **Eliminar** o **Imputar** (rellenar).

**Estrategia 1: Eliminar (`.dropna()`)**

Es la opción más sencilla, pero también la más peligrosa. Si eliminamos filas, perdemos información valiosa.

* **Eliminar filas:** Si una fila tiene *cualquier* valor NaN. Esto suele ser demasiado agresivo.

In [9]:
# Vemos cuántas filas nos quedarían si eliminamos todas las que tengan AL MENOS UN NaN
print("Filas originales:", df_titanic.shape[0])
print("Filas después de dropna():", df_titanic.dropna().shape[0])

Filas originales: 891
Filas después de dropna(): 182


¡Pasamos de 891 filas a 182! Hemos perdido casi el 80% de los datos. Esto se debe a que la columna `deck` tiene muchísimos NaN. 

* **Eliminar columnas:** Es una buena idea si la columna está casi vacía y no es útil. La columna `deck` solo tiene 203 valores de 891. Probablemente es mejor eliminarla.

`axis=1` se refiere a las columnas. `how='all'` significa "elimina solo si *todos* los valores de la columna son NaN". El umbral `thresh` (threshold) es más útil: "mantén solo las columnas que tengan *al menos* X valores no nulos".

In [8]:
# Creamos una copia para no modificar el original
df_limpio = df_titanic.copy()

# Eliminamos la columna 'deck' (axis=1)
df_limpio = df_limpio.drop('deck', axis=1)

df_limpio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  embark_town  889 non-null    object  
 12  alive        891 non-null    object  
 13  alone        891 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(5)
memory usage: 79.4+ KB


¡Genial! Nos hemos deshecho de `deck`, pero aún tenemos NaN en `age` y `embarked`. No queremos eliminar esas filas.

**Estrategia 2: Imputar o Rellenar (`.fillna()`)**

Rellenamos los NaN con un valor. La pregunta es: ¿con qué valor?

* Para `age` (numérica): Una estrategia común es usar la **media** o la **mediana** de la edad. La mediana (el valor central) suele ser mejor si hay valores atípicos (edades muy altas o muy bajas).
* Para `embarked` (categórica): La estrategia común es usar la **moda** (el valor más frecuente).

In [9]:
# 1. Rellenar 'age' con la mediana
mediana_edad = df_limpio['age'].median()
print(f"La mediana de edad es: {mediana_edad}")
df_limpio['age'] = df_limpio['age'].fillna(mediana_edad)

# 2. Rellenar 'embarked' con la moda
# .mode() devuelve una Serie (porque podría haber varias modas), 
# así que seleccionamos la primera con [0].
moda_embarked = df_limpio['embarked'].mode()[0]
print(f"El puerto de embarque más frecuente es: {moda_embarked}")
df_limpio['embarked'] = df_limpio['embarked'].fillna(moda_embarked)

# 3. Comprobamos el resultado
print("\n--- Información post-limpieza ---")
df_limpio.info()

La mediana de edad es: 28.0
El puerto de embarque más frecuente es: S

--- Información post-limpieza ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          891 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     891 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  embark_town  889 non-null    object  
 12  alive        891 non-null    object  
 13  alone        891 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(5)
memory usage: 79.4+ KB


¡Perfecto! Ahora tenemos 891 entradas no nulas en todas las columnas que nos interesan.

#### 2. Limpieza de Datos Duplicados

Los datos duplicados pueden sesgar nuestros análisis (por ejemplo, contando dos veces la misma venta). Pandas nos ayuda a encontrarlos y eliminarlos.

Primero, vamos a crear un pequeño DataFrame de ejemplo para ver cómo funciona.

In [10]:
datos_dup = pd.DataFrame({
    'id_cliente': [1, 2, 3, 2, 4],
    'email': ['a@mail.com', 'b@mail.com', 'c@mail.com', 'b@mail.com', 'd@mail.com']
})

print("Datos con duplicados:")
datos_dup

Datos con duplicados:


,id_cliente,email
0,1,a@mail.com
1,2,b@mail.com
2,3,c@mail.com
3,2,b@mail.com
4,4,d@mail.com


In [11]:
# .duplicated() nos devuelve una Serie booleana que es True para cada fila duplicada
print("¿Qué filas están duplicadas?")
print(datos_dup.duplicated())

¿Qué filas están duplicadas?
0    False
1    False
2    False
3     True
4    False
dtype: bool


Pandas marca la fila 3 (índice 3) como duplicada porque es idéntica a la fila 1 (índice 1).

Para eliminarlas, usamos `.drop_duplicates()`.

In [12]:
df_sin_duplicados = datos_dup.drop_duplicates()
print("Datos sin duplicados:")
df_sin_duplicados

Datos sin duplicados:


,id_cliente,email
0,1,a@mail.com
1,2,b@mail.com
2,3,c@mail.com
4,4,d@mail.com


A veces, una fila no es un duplicado exacto, pero un campo clave sí lo es (p.ej., el email). Podemos usar el parámetro `subset` para buscar duplicados basándonos solo en columnas específicas.

In [13]:
datos_dup_email = pd.DataFrame({
    'id_cliente': [1, 2, 3, 4],
    'email': ['a@mail.com', 'b@mail.com', 'c@mail.com', 'b@mail.com']
})

print("Datos con email duplicado:")
print(datos_dup_email)

# Eliminamos filas basándonos solo en la columna 'email', manteniendo la primera aparición
df_email_unico = datos_dup_email.drop_duplicates(subset=['email'], keep='first')
print("\nDatos con email único:")
print(df_email_unico)

Datos con email duplicado:
   id_cliente       email
0           1  a@mail.com
1           2  b@mail.com
2           3  c@mail.com
3           4  b@mail.com

Datos con email único:
   id_cliente       email
0           1  a@mail.com
1           2  b@mail.com
2           3  c@mail.com


---

### Ejercicios del Bloque 1 

**Dataset:** Para estos ejercicios, usaremos el dataset `diamonds` (diamantes), también de la librería `seaborn`.

**Ejercicio 1: Análisis Preliminar**
1.  Carga el dataset `diamonds` de `seaborn` en un DataFrame llamado `df_diamantes`.
2.  Muestra la forma (`.shape`) del DataFrame.
3.  Muestra las 5 primeras filas.
4.  Utiliza `.info()` para identificar qué columnas tienen datos faltantes (si las hay) y qué tipos de datos tiene cada columna.
5.  Utiliza `.describe()` para obtener las estadísticas de las columnas numéricas. ¿Observas algo extraño en los valores `min` de las columnas `x`, `y`, o `z` (dimensiones)?

**Ejercicio 2: Limpieza de Duplicados**
1.  Comprueba cuántas filas duplicadas exactas hay en `df_diamantes`. 
2.  Crea un nuevo DataFrame llamado `df_diamantes_limpio` que sea una copia del original, pero sin las filas duplicadas.
3.  Verifica la forma del nuevo DataFrame para confirmar que se han eliminado las filas.

**Ejercicio 3: Limpieza de Datos Faltantes y Atípicos**
1.  Basándote en el análisis del Ejercicio 1.5, has visto que algunos diamantes tienen dimensiones `x`, `y` o `z` iguales a 0, lo cual es físicamente imposible. Estos son datos erróneos, no 

---

## Bloque 2: Estandarización y Transformación de Datos

**Objetivos del Bloque:**
1.  **Transformar** datos usando mapeos y funciones.
2.  **Estandarizar y Normalizar** datos numéricos.
3.  **Discretizar** variables continuas en intervalos.
4.  **Identificar y convertir** variables categóricas a numéricas.

Continuaremos usando el DataFrame `df_limpio` (Titanic) del bloque anterior.

In [14]:
# Recordatorio de nuestros datos limpios de NaN
df_limpio.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,Southampton,no,True


### Transformación de Datos

A menudo necesitamos modificar los valores de una columna. Por ejemplo, la columna `sex` es textual ('male', 'female'). Para muchos modelos, necesitaremos que sea numérica (0 y 1).

#### 1. Mapeo de Valores: `.map()`
El método `.map()` nos permite sustituir cada valor de una Serie por otro valor, usando un diccionario (un "mapeo") [cite: 2007-2008].

In [15]:
# Creamos una nueva columna 'sex_num' con los valores mapeados
mapa_sexo = {'male': 0, 'female': 1}
df_limpio['sex_num'] = df_limpio['sex'].map(mapa_sexo)

df_limpio[['sex', 'sex_num']].head()

,sex,sex_num
0,male,0
1,female,1
2,female,1
3,female,1
4,male,0


#### 2. Aplicar Funciones: `.apply()`
`.apply()` es más potente que `.map()`. Permite aplicar una función a cada fila o columna. La usaremos aquí para crear una nueva característica (Feature Engineering).

Vamos a crear una columna `family_size` (tamaño de la familia) sumando `sibsp` (hermanos/cónyuges) y `parch` (padres/hijos) + 1 (la propia persona).

In [17]:
# Definimos una función (o podemos usar lambda)
def calcular_tam_familia(fila):
    return fila['sibsp'] + fila['parch'] + 1

# axis=1 significa aplicar la función por filas, por lo que la función recibe una fila entera en cada llamada.
# En este caso, con axis=1 la función calcular_tam_familia puede combinar o evaluar múltiples columnas de esa fila 
# para calcular un nuevo valor (tamaño de familia basado las columnas 'sibsp' y 'parch').
# Si el valor de axis fuera 0, la función se aplicaría por columnas(pasaria una columna entera a la función), lo que no tendría sentido en este caso.
df_limpio['family_size'] = df_limpio.apply(calcular_tam_familia, axis=1)

df_limpio[['sibsp', 'parch', 'family_size']].head()

,sibsp,parch,family_size
0,1,0,2
1,1,0,2
2,0,0,1
3,1,0,2
4,0,0,1


### Normalización y Estandarización

Este es un concepto clave. Muchos algoritmos de machine learning (como regresiones o redes neuronales) funcionan mucho peor si las variables numéricas tienen escalas muy diferentes.

En nuestro dataset, `age` va de 0.4 a 80, pero `fare` (tarifa) va de 0 a 512. Esta diferencia de escala puede confundir al algoritmo.

**Dos técnicas principales:**
1.  **Normalización (Min-Max Scaling):** Re-escala los datos para que estén en un rango fijo, generalmente entre 0 y 1. ([API](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html))   
La fórmula es:   

$$
X_{scaled} = \frac{X - min(X)}{(max(X) - min(X))}*(max-min)+min
$$

En nuestro ejemplo vamos a normalizar la columna 'fare'. Tenemos en consideración lo siguiente:
- La columna 'fare' representa tarifas pagadas, que pueden tener valores muy dispersos, con un rango amplio y posiblemente valores atípicos elevados.
- La normalización escala estos datos a un rango fijo (de 0 a 1). Esto permite mantener la proporción relativa entre tarifas y que se procesen esos datos **sin que la magnitud de 'fare' domine** la influencia del cálculo del modelo.
- Al normalizar, **se conserva** la forma de la distribución pero todos los valores quedan en una escala comparable y acotada.   


2.  **Estandarización (Z-score Scaling):** Re-escala los datos para que tengan una meda (μ) de 0 y una desviación estándar (σ) de 1. ([API](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html))   
La fórmula es:   
$$
 z = \frac{(X - \mu)}{\sigma}.
$$

En nuestro ejemplo, la columna 'age' es la que procedemos a estandarizar por lo siguiente:    

- La edad es una variable con distribución 'normal', o simétrica, con media y desviación estándar bien definidas.
- Estandarizar transforma estos datos para que tengan media cero y desviación estándar uno, facilitando el procesamiento estadístico y la comparación entre variables.
- Al centrar y escalar, se reduce la influencia de la escala original y se facilita la interpretación y convergencia en los modelos.   

TABLA RESUMEN

|Técnica|Uso|Ventajas/Inconvenientes|
|-------|---|-----------------------|
|Normalización|Cuando los datos tienen rangos muy diferentes o valores atípicos marcados; o se necesita un rango fijo|Mantiene proporciones; útil para algoritmos sensibles al rango; puede ser afectada por outliers|
|Estandarización|Cuando los datos se aproximan a una distribución normal o se requiere centrar datos en media cero|Mejora estabilidad numérica; reduce influencia de escala; menos sensible a outliers que normalización|
   

Usaremos la librería `scikit-learn` para aplicar estos métodos de transformación.

In [19]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# 1. Estandarización (Z-score) para 'age'
scaler_std = StandardScaler()
# .fit_transform() aprende la media y desv. est. y luego transforma los datos
# [:, np.newaxis] convierte la Serie 1D en un array 2D, que es lo que scikit-learn espera
df_limpio['age_standard'] = scaler_std.fit_transform(df_limpio[['age']])

# 2. Normalización (Min-Max) para 'fare'
scaler_norm = MinMaxScaler()
df_limpio['fare_normalized'] = scaler_norm.fit_transform(df_limpio[['fare']])

# Comprobamos los resultados
print("--- Datos escalados ---")
print(df_limpio[['age', 'age_standard', 'fare', 'fare_normalized']].head())

print("\n--- Verificación de la Estandarización ---")
print(f"Media de age_standard: {df_limpio['age_standard'].mean():.2f}")
print(f"Desv. Est. de age_standard: {df_limpio['age_standard'].std():.2f}")

print("\n--- Verificación de la Normalización ---")
print(f"Mínimo de fare_normalized: {df_limpio['fare_normalized'].min():.2f}")
print(f"Máximo de fare_normalized: {df_limpio['fare_normalized'].max():.2f}")

--- Datos escalados ---
    age  age_standard     fare  fare_normalized
0  22.0     -0.565736   7.2500         0.014151
1  38.0      0.663861  71.2833         0.139136
2  26.0     -0.258337   7.9250         0.015469
3  35.0      0.433312  53.1000         0.103644
4  35.0      0.433312   8.0500         0.015713

--- Verificación de la Estandarización ---
Media de age_standard: 0.00
Desv. Est. de age_standard: 1.00

--- Verificación de la Normalización ---
Mínimo de fare_normalized: 0.00
Máximo de fare_normalized: 1.00


### Aplicación de intervalos (Discretización)

A veces es útil convertir una variable continua (como `age`) en una variable categórica (como 'Niño', 'Adulto', 'Senior'). A esto se le llama **discretización** o **binning**.

Pandas nos da dos funciones para esto: `pd.cut` (cortar por rangos) y `pd.qcut` (cortar por cuantiles).

In [20]:
# Usaremos pd.cut para crear grupos de edad
# 1. Definimos los "cortes" (bins)
bins_edad = [0, 12, 18, 35, 60, 80] # Cortes: 0-12, 13-18, 19-35, 36-60, 61-80

# 2. Definimos las etiquetas para esos cortes
labels_edad = ['Infantil', 'Adolescente', 'Joven Adulto', 'Adulto', 'Senior']

# 3. Aplicamos la función
df_limpio['grupo_edad'] = pd.cut(df_limpio['age'], 
                               bins=bins_edad, 
                               labels=labels_edad, 
                               right=True) # right=True significa que el intervalo incluye el valor derecho (ej: (12, 18])

print(df_limpio[['age', 'grupo_edad']].head())

# Veamos el conteo de cada nuevo grupo
print("\nConteo por grupo de edad:")
print(df_limpio['grupo_edad'].value_counts())

    age    grupo_edad
0  22.0  Joven Adulto
1  38.0        Adulto
2  26.0  Joven Adulto
3  35.0  Joven Adulto
4  35.0  Joven Adulto

Conteo por grupo de edad:
grupo_edad
Joven Adulto    535
Adulto          195
Adolescente      70
Infantil         69
Senior           22
Name: count, dtype: int64


### Identificar y Modificar Variables Categóricas

**Identificación:** Ya lo hemos estado haciendo. Columnas como `sex`, `embarked`, `pclass` y nuestro nuevo `grupo_edad` son categóricas. Representan una cualidad o grupo, no una cantidad.

**Modificación a Cuantitativas:** ¿Por qué convertir texto en números? Porque los modelos matemáticos (como las regresiones) no entienden "Southampton". Necesitan números.

Tenemos dos tipos de variables categóricas:
1.  **Ordinales:** Tienen un orden lógico (ej: 'Malo' < 'Bueno' < 'Excelente'). Las mapeamos a números que respeten ese orden (ej: 0, 1, 2).
2.  **Nominales:** No tienen un orden lógico (ej: 'Madrid', 'Barcelona', 'Valencia').

Ya vimos cómo convertir `sex` (Nominal, 2 categorías) usando `.map()`.

¿Qué pasa con `embarked` (Nominal, 3 categorías)? ¿Podríamos hacer `{'S': 0, 'C': 1, 'Q': 2}`? 

**¡NO!** Esto implicaría un orden falso (que C(1) es "más" que S(0), y Q(2) es "más" que C(1)), lo cual es incorrecto y confundiría al modelo. 

La solución se llama **Codificación One-Hot** (o variables *dummy*). Creamos una nueva columna binaria (0/1) por cada categoría.

In [21]:
# pd.get_dummies() hace esto automáticamente por nosotros
df_dummies_embarked = pd.get_dummies(df_limpio['embarked'], prefix='puerto')

print("Columnas Dummy para 'embarked':")
print(df_dummies_embarked.head())

# Ahora, unimos estas nuevas columnas a nuestro DataFrame principal
# (Y solemos eliminar la columna original 'embarked')
df_final = pd.concat([df_limpio, df_dummies_embarked], axis=1)
df_final = df_final.drop('embarked', axis=1)

print("\nDataFrame final con Dummies:")
df_final.head()

Columnas Dummy para 'embarked':
   puerto_C  puerto_Q  puerto_S
0     False     False      True
1      True     False     False
2     False     False      True
3     False     False      True
4     False     False      True

DataFrame final con Dummies:


,survived,pclass,sex,age,sibsp,parch,fare,class,who,adult_male,embark_town,alive,alone,sex_num,family_size,age_standard,fare_normalized,grupo_edad,puerto_C,puerto_Q,puerto_S
0,0,3,male,22.0,1,0,7.2500,Third,man,True,Southampton,no,False,0,2,-0.565736,0.014151,Joven Adulto,False,False,True
1,1,1,female,38.0,1,0,71.2833,First,woman,False,Cherbourg,yes,False,1,2,0.663861,0.139136,Adulto,True,False,False
2,1,3,female,26.0,0,0,7.9250,Third,woman,False,Southampton,yes,True,1,1,-0.258337,0.015469,Joven Adulto,False,False,True
3,1,1,female,35.0,1,0,53.1000,First,woman,False,Southampton,yes,False,1,2,0.433312,0.103644,Joven Adulto,False,False,True
4,0,3,male,35.0,0,0,8.0500,Third,man,True,Southampton,no,True,0,1,0.433312,0.015713,Joven Adulto,False,False,True


¡Y ya está! Nuestro DataFrame `df_final` (excepto por las columnas de texto que dejamos, como `name`) está casi listo para ser usado en un modelo de Machine Learning. Hemos limpiado NaNs, eliminado duplicados, transformado variables, estandarizado las numéricas y codificado las categóricas.

---

### Ejercicios del Bloque 2

**Dataset:** Continuaremos con el DataFrame `df_diamantes_limpio` del Bloque 1.

**Ejercicio 1: Transformación y Estandarización**
1.  Hay 3 columnas con dimensiones: `x`, `y`, `z`. Crea una nueva columna llamada `volumen` que sea `x * y * z`.
2.  Crea una nueva columna llamada `densidad` que sea `carat / volumen`. Es posible que obtengas valores `inf` (infinitos) si `volumen` es 0 (por los datos erróneos que limpiaste). Reemplaza estos valores `inf` por `NaN` y luego rellénalos con 0.
3.  Estandariza (Z-score) las columnas `carat`, `depth`, `table` y `price`. Almacena los resultados en nuevas columnas (ej: `carat_std`, `price_std`).

**Ejercicio 2: Discretización**
1.  Usando `pd.qcut` (corte por cuantiles), divide la columna `price` en 4 contenedores (cuartiles) con las etiquetas: `['Barato', 'Medio', 'Caro', 'Muy Caro']`. Guarda esto en una nueva columna llamada `rango_precio`.
2.  Muestra el conteo de valores (`.value_counts()`) de esta nueva columna `rango_precio`.

**Ejercicio 3: Codificación Categórica**
1.  El dataset `diamonds` tiene 3 columnas categóricas nominales: `cut`, `color`, y `clarity`.
2.  Crea un nuevo DataFrame llamado `df_diamantes_dummies` que sea el resultado de aplicar `pd.get_dummies()` a `df_diamantes_limpio`, pero **solo** sobre estas 3 columnas.
3.  Muestra las 5 primeras filas del nuevo `df_diamantes_dummies`. ¿Cuántas columnas tiene ahora?

**Ejercicio 4 (Bonus):**
1.  La columna `cut` (corte) en realidad es **Ordinal**. Sus valores (según la documentación de `seaborn`) son: `['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']`.
2.  Crea una nueva columna `cut_ordinal` donde mapees 'Fair' a 0, 'Good' a 1, y así sucesivamente hasta 'Ideal' a 4.
3.  Compara esto con el resultado de `pd.get_dummies()` para `cut`. ¿Cuándo usarías un método sobre el otro?